In [1]:
import spatialdata
import spatialdata_plot
import dask_image.imread
import dask.array as da
import napari_spatialdata
import matplotlib.pyplot as plt
from matplotlib import patches
import geopandas as gpd
import anndata as ad
import numpy as np
import anndata as ad

import sys, os
sys.path.append(os.path.abspath("../src/opendvp/"))
import anndata_utils

for package in [spatialdata, napari_spatialdata]:
    print(f"{package.__name__}: {package.__version__}")

#paths
path_to_mIF_image = "/Volumes/Extreme SSD/P26_BCCN/P26E07_E08_E09_SOPA_Gates/processed_images/991.ome.tif"
path_to_HE_image = "/Volumes/Extreme SSD/P26_BCCN/P26_HE_staining/240624_SF_P26_mDVP_991_HE.ome.tiff"
path_to_mask = "/Volumes/Extreme SSD/P26_BCCN/P26E07_E08_E09_SOPA_Gates/expanded-masks/991.tif"
path_to_quant = "/Volumes/Extreme SSD/P26_BCCN/P26E07_E08_E09_SOPA_Gates/quantification/991.csv"

/opt/homebrew/Caskroom/mambaforge/base/envs/spatialdata/lib/python3.12/site-packages/dask/dataframe/__init__.py:31: FutureWarning: The legacy Dask DataFrame implementation is deprecated and will be removed in a future version. Set the configuration option `dataframe.query-planning` to `True` or None to enable the new Dask Dataframe implementation and silence this warning.
  warnings.warn(


spatialdata: 0.3.0
napari_spatialdata: 0.5.5


In [2]:
! mamba list

# packages in environment at /opt/homebrew/Caskroom/mambaforge/base/envs/spatialdata:
#
# Name                    Version                   Build  Channel
aiobotocore               2.5.4                    pypi_0    pypi
aiohappyeyeballs          2.4.6              pyhd8ed1ab_0    conda-forge
aiohttp                   3.11.13         py312h998013c_0    conda-forge
aioitertools              0.12.0                   pypi_0    pypi
aiosignal                 1.3.2              pyhd8ed1ab_0    conda-forge
alabaster                 1.0.0              pyhd8ed1ab_1    conda-forge
anndata                   0.11.3             pyhd8ed1ab_0    conda-forge
annotated-types           0.7.0              pyhd8ed1ab_1    conda-forge
aom                       3.9.1                h7bae524_0    conda-forge
app-model                 0.3.0              pyhd8ed1ab_0    conda-forge
appdirs                   1.4.4              pyhd8ed1ab_1    conda-forge
appnope                   0.1.4              pyhd8ed1ab_

# Create sdata

In [3]:
sdata = spatialdata.SpatialData()

## images and mask

In [4]:
#parse in images

sdata['mIF'] = spatialdata.models.Image2DModel.parse(
    data=dask_image.imread.imread(path_to_mIF_image),
    dims=['c', 'y', 'x'],
    transformations=None,
    scale_factors=[3,3,3],
    )

In [5]:
sdata.write("../data/sdata/sdata_991.zarr")

INFO     The Zarr backing store has been changed from None the new file path: ../data/sdata/sdata_991.zarr         


In [6]:
#parse labels
sdata['mask'] = spatialdata.models.Labels2DModel.parse(
    data=da.squeeze(dask_image.imread.imread(path_to_mask)),
    dims=['y','x'],
    scale_factors=[3,3,3],
    )

In [7]:
sdata.write_element("mask")

In [8]:
sdata['mask_polygons'] = spatialdata.to_polygons(sdata['mask'])

In [9]:
# parse polygons index
sdata['mask_polygons']['label'] = sdata['mask_polygons']['label'].astype('int')
sdata['mask_polygons'].index = sdata['mask_polygons'].index.astype('int')

In [10]:
sdata.write_element("mask_polygons")

## Link polygons with imaging table

In [11]:
adata = ad.read_h5ad("/Users/jnimoca/Jose_BI/4_Projects/P26_BCCN/P26E10_E11_ImageAnalysis/data/perSample/991_992/1_spatial_LDA/20250303_1654_1_spatial_LDA_adata.h5ad")

In [12]:
adata_991 = adata[adata.obs.imageid == 991]

In [13]:
def ensure_one_based_index(adata, cellid_col="CellID"):
    """
    Ensures the specified CellID column and index are 1-based.
    Converts data to integers if needed.
    
    Parameters:
    - adata: AnnData object
    - cellid_col: str, name of the column with cell IDs (default: "CellID")
    
    Returns:
    - adata: updated AnnData object
    """
    
    # Check if the column exists
    if cellid_col not in adata.obs.columns:
        raise ValueError(f"Column '{cellid_col}' not found in adata.obs.")
    
    # Ensure the CellID column and index are integers
    if not np.issubdtype(adata.obs[cellid_col].dtype, np.integer):
        adata.obs[cellid_col] = adata.obs[cellid_col].astype(int)

    if not np.issubdtype(adata.obs.index.dtype, np.integer):
        adata.obs.index = adata.obs.index.astype(int)
    
    # Check if both are 0-based and increment if needed
    if (adata.obs[cellid_col].min() == 0) and (adata.obs.index.min() == 0):
        adata.obs[cellid_col] += 1
        adata.obs.index += 1
        print(f"✅ Incremented '{cellid_col}' and index to 1-based numbering.")
    else:
        print("⏭️ Skipping increment: CellID or index is not 0-based.")
    
    return adata

In [14]:
ensure_one_based_index(adata_991)

/var/folders/x7/grkjlk8s223dy6234rnz1885mxz2_6/T/ipykernel_7681/2157123882.py:23: ImplicitModificationWarning: Trying to modify index of attribute `.obs` of view, initializing view as actual.
  adata.obs.index = adata.obs.index.astype(int)


⏭️ Skipping increment: CellID or index is not 0-based.


/opt/homebrew/Caskroom/mambaforge/base/envs/spatialdata/lib/python3.12/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


AnnData object with n_obs × n_vars = 610182 × 8
    obs: 'CellID', 'Y_centroid', 'X_centroid', 'Area', 'MajorAxisLength', 'MinorAxisLength', 'Eccentricity', 'Orientation', 'Extent', 'Solidity', 'artefact', 'Area_filter_nottoobig', 'Area_filter_nottoolow', 'Area_filter', 'DAPI_ratio', 'DAPI_ratio_pass_nottoolow', 'DAPI_ratio_pass_nottoohigh', 'DAPI_ratio_pass', 'filtering', 'imageid', 'phenotype', 'cell_id', 'spatial_lda_knn7_kmeans_k7', 'spatial_lda_knn14_kmeans_k7', 'spatial_lda_knn21_kmeans_k7', 'spatial_lda_knn30_kmeans_k7', 'spatial_lda_knn40_kmeans_k7', 'spatial_lda_knn50_kmeans_k7'
    uns: 'spatial_lda_knn14', 'spatial_lda_knn14_probability', 'spatial_lda_knn21', 'spatial_lda_knn21_probability', 'spatial_lda_knn30', 'spatial_lda_knn30_probability', 'spatial_lda_knn40', 'spatial_lda_knn40_probability', 'spatial_lda_knn50', 'spatial_lda_knn50_probability', 'spatial_lda_knn7', 'spatial_lda_knn7_probability'
    layers: 'log'

In [15]:
adata_991.obs["shapes"] = "mask_polygons"
adata_991.obs["shapes"] = adata_991.obs["shapes"].astype("category")
sdata["imaging_991"] = spatialdata.models.TableModel.parse(adata_991, region = "mask_polygons", region_key = "shapes", instance_key = "CellID")

In [16]:
sdata.write_element("imaging_991")

# QuPath contours

In [17]:
gdf = gpd.read_file("/Users/jnimoca/Jose_BI/4_Projects/P26_BCCN/P26E18_ProteomicAnalysis/data/geojsons/clean_gdf.geojson")
gdf.index = gdf.new_class_name # this column has matching ids with proteomic table
sdata['primary_lmd_contours'] = spatialdata.models.ShapesModel.parse(gdf[gdf.biopsy_type == "Primary"])
sdata['relapse_lmd_contours'] = spatialdata.models.ShapesModel.parse(gdf[gdf.biopsy_type == "Relapse"])

In [18]:
sdata.write_element("primary_lmd_contours")

In [19]:
sdata.write_element("relapse_lmd_contours")

# Proteomic data

In [20]:
proteins = ad.read_h5ad("/Users/jnimoca/Jose_BI/4_Projects/P26_BCCN/P26E18_ProteomicAnalysis/data/checkpoints/7_ttest_RCN1/20250318_1121_7_ttest_RCN1_adata.h5ad")

In [21]:
proteins.obs["shapes"] = "primary_lmd_contours"
proteins.obs["shapes"] = proteins.obs["shapes"].astype("category")
proteins.obs['class_name'] = proteins.obs["class_name"].astype("str")
sdata["proteins"] = spatialdata.models.TableModel.parse(proteins, region = "primary_lmd_contours", region_key = "shapes", instance_key = "class_name")

In [22]:
sdata.write_element("proteins")

In [2]:
sdata = spatialdata.read_zarr("../data/sdata/sdata_991.zarr")

/opt/homebrew/Caskroom/mambaforge/base/envs/spatialdata/lib/python3.12/site-packages/zarr/creation.py:614: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)


In [ ]:
napari_spatialdata.Interactive(sdata).run()

/opt/homebrew/Caskroom/mambaforge/base/envs/spatialdata/lib/python3.12/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
2025-03-19 09:31:43.238 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2025-03-19 09:31:43.238 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
/opt/homebrew/Caskroom/mambaforge/base/envs/spatialdata/lib/python3.12/site-packages/anndata/_core/anndata.py:1756: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
2025-03-19 09:31:50.289 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2025-03-19 09:31:50.292 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2025-03-19 09:31:50.325 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating la

# Crop it

In [3]:
# bb_xmin = 35000
# bb_ymin = 19000
# bb_w = 3000
# bb_h = 3000
# bb_xmax = bb_xmin + bb_w
# bb_ymax = bb_ymin + bb_h

bb_xmin, bb_xmax, bb_ymin, bb_ymax = 35000,38000, 19500, 23000

cropped_sdata = sdata.query.bounding_box(
    axes=["x", "y"],
    min_coordinate=[bb_xmin, bb_ymin],
    max_coordinate=[bb_xmax, bb_ymax],
    target_coordinate_system="global",
    filter_table=True,
)

In [11]:
cropped_sdata.write("../data/sdata/crop_sdata_991.zarr")

INFO     The SpatialData object is not self-contained (i.e. it contains some elements that are Dask-backed from    
         locations outside ../data/sdata/crop_sdata_991.zarr). Please see the documentation of                     
         `is_self_contained()` to understand the implications of working with SpatialData objects that are not     
         self-contained.                                                                                           
INFO     The Zarr backing store has been changed from None the new file path: ../data/sdata/crop_sdata_991.zarr    


In [9]:
cropped_sdata

SpatialData object
├── Images
│     └── 'mIF': DataTree[cyx] (15, 4000, 4000), (15, 1333, 1334), (15, 444, 445), (15, 148, 148)
├── Labels
│     └── 'mask': DataTree[yx] (4000, 4000), (1333, 1334), (444, 445), (148, 148)
├── Shapes
│     ├── 'mask_polygons': GeoDataFrame shape: (8947, 2) (2D shapes)
│     ├── 'primary_lmd_contours': GeoDataFrame shape: (6, 8) (2D shapes)
│     └── 'relapse_lmd_contours': GeoDataFrame shape: (11, 8) (2D shapes)
└── Tables
      ├── 'imaging_991': AnnData (7718, 8)
      └── 'proteins': AnnData (3, 4766)
with coordinate systems:
    ▸ 'global', with elements:
        mIF (Images), mask (Labels), mask_polygons (Shapes), primary_lmd_contours (Shapes), relapse_lmd_contours (Shapes)

In [20]:
napari_spatialdata.Interactive(cropped_sdata).run()

In [17]:
cropped_sdata.write("../data/sdata/crop_mIF_fig1.zarr")

INFO     The SpatialData object is not self-contained (i.e. it contains some elements that are Dask-backed from    
         locations outside ../data/sdata/crop_mIF_fig1.zarr). Please see the documentation of `is_self_contained()`
         to understand the implications of working with SpatialData objects that are not self-contained.           
INFO     The Zarr backing store has been changed from None the new file path: ../data/sdata/crop_mIF_fig1.zarr     
